## Data Reading

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df=spark.read.format('parquet').load('abfss://bronze2026@storagespotifyproject20.dfs.core.windows.net/customers')

## Drop Column

In [0]:
df=df.drop('_rescued_data')

## Create a new col to capture the domains

In [0]:
df=df.withColumn('Domain',right('email',length('email') - instr('email', '@')))

## Aggration function

In [0]:
# df.groupBy('Domain').agg(count('customer_id').alias('total_customer')).orderBy('total_customer').display()

In [0]:
# df.where("Domain in ('gmail.com','yahoo.com','hotmail.com') ").display()

## Full Name creation

In [0]:
df=df.withColumn('full_name',concat(col('first_name'),lit(' '),col('last_name'))).drop('first_name','last_name')

##Data Writing

In [0]:
df.write.format('delta').mode('append').save("abfss://silver2026@storagespotifyproject20.dfs.core.windows.net/Customers")

## Creating Table in silver layer

In [0]:
%sql
create table if not exists databricks_cata.silver.Customers_silver
using delta
location 'abfss://silver2026@storagespotifyproject20.dfs.core.windows.net/Customers'
